In [ ]:
import ast


In [ ]:
net_features = []
with open("nets.txt", "r") as f:
    for line in f:
        net = ast.literal_eval(line.strip())  # safely parse dictionary from string
        net_features.append(net)

FileNotFoundError: [Errno 2] No such file or directory: 'nets.txt'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
net_features = []
with open('/content/drive/MyDrive/nets.txt', 'r') as f:
    for line in f:
        net = ast.literal_eval(line.strip())  # safely parse dictionary from string
        net_features.append(net)

In [ ]:
node_features = {}
for net in net_features:
    driver = net['driver']
    node_features[driver['id']] = [
        driver['x'], driver['y'], int(driver['is_fixed']), driver['area']
    ]
    for sink in net['sinks']:
        node_features[sink['id']] = [
            sink['x'], sink['y'], int(sink['is_fixed']), sink['area']
        ]


In [ ]:
sorted_node_ids = sorted(node_features.keys())
node_id_to_index = {nid: idx for idx, nid in enumerate(sorted_node_ids)}

# Node feature matrix
import torch
X = torch.tensor([node_features[nid] for nid in sorted_node_ids], dtype=torch.float)

# Create hyperedges
hyperedges = []
for net in net_features:
    node_ids = [net['driver']['id']] + [sink['id'] for sink in net['sinks']]
    edge = [node_id_to_index[nid] for nid in node_ids]
    hyperedges.append(edge)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class HyperGNN(nn.Module):
    def __init__(self, in_dim, hidden_dim):
        super(HyperGNN, self).__init__()
        self.fc1 = nn.Linear(in_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, 2)  # Output: (x, y) positions

    def forward(self, x):
        x = F.relu(self.fc1(x))
        pos = self.fc2(x)
        return pos  # Shape: [num_nodes, 2]

In [ ]:
def smooth_hpwl(positions, hyperedges, alpha=10.0):
    """
    Args:
        positions: Tensor of shape (num_nodes, 2) with (x, y) positions
        hyperedges: List of lists, each sublist is indices of nodes in that net
        alpha: Smoothness parameter for log-sum-exp

    Returns:
        Total smooth HPWL over all nets
    """
    total_hpwl = 0.0
    for net in hyperedges:
        pins = positions[net]  # Shape: (num_pins_in_net, 2)
        x = pins[:, 0]
        y = pins[:, 1]

        max_x = (1.0 / alpha) * torch.logsumexp(alpha * x, dim=0)
        min_x = -(1.0 / alpha) * torch.logsumexp(-alpha * x, dim=0)
        max_y = (1.0 / alpha) * torch.logsumexp(alpha * y, dim=0)
        min_y = -(1.0 / alpha) * torch.logsumexp(-alpha * y, dim=0)

        hpwl = (max_x - min_x) + (max_y - min_y)
        total_hpwl += hpwl

    return total_hpwl





In [ ]:

def compute_hpwl(positions, hyperedges):
    hpwl = 0.0
    for edge in hyperedges:
        x_coords = positions[edge, 0]
        y_coords = positions[edge, 1]
        hpwl += (x_coords.max() - x_coords.min()) + (y_coords.max() - y_coords.min())
    return hpwl

def compute_density_loss(positions, areas, grid_size):
    total_area = areas.sum()
    return total_area / grid_size


In [ ]:
def train(model, X, hyperedges, areas, grid_size, num_epochs=500, lr=0.05):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    for epoch in range(num_epochs):
        model.train()
        positions = model(X)

        hpwl_loss = smooth_hpwl(positions, hyperedges)
        density_loss = compute_density_loss(positions, areas, grid_size)

        loss = hpwl_loss + 0.1 * density_loss  # Weight density as secondary

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if epoch % 10 == 0:
            print(f"Epoch {epoch}: HPWL={hpwl_loss.item():.2f}, Density={density_loss.item():.4f}")

In [ ]:
# Setup and run
in_dim = X.shape[1]
hidden_dim = 64
model = HyperGNN(in_dim, hidden_dim)

areas = torch.tensor([f[-1] for f in X], dtype=torch.float)
grid_size = 1000 * 1000  # Example total area

train(model, X, hyperedges, areas, grid_size)

Epoch 0: HPWL=412501440.00, Density=2600.4163
Epoch 10: HPWL=264272416.00, Density=2600.4163
Epoch 20: HPWL=41239008.00, Density=2600.4163
Epoch 30: HPWL=3341510.75, Density=2600.4163
Epoch 40: HPWL=75265.17, Density=2600.4163
Epoch 50: HPWL=36900.07, Density=2600.4163
Epoch 60: HPWL=33487.84, Density=2600.4163
Epoch 70: HPWL=19921.80, Density=2600.4163
Epoch 80: HPWL=40710.61, Density=2600.4163
Epoch 90: HPWL=44928.56, Density=2600.4163
Epoch 100: HPWL=8849.26, Density=2600.4163
Epoch 110: HPWL=41713.75, Density=2600.4163
Epoch 120: HPWL=60301.87, Density=2600.4163
Epoch 130: HPWL=59773.59, Density=2600.4163
Epoch 140: HPWL=90848.37, Density=2600.4163
Epoch 150: HPWL=46783.57, Density=2600.4163
Epoch 160: HPWL=52499.32, Density=2600.4163
Epoch 170: HPWL=44910.65, Density=2600.4163
Epoch 180: HPWL=15058.61, Density=2600.4163
Epoch 190: HPWL=36156.08, Density=2600.4163
Epoch 200: HPWL=34081.33, Density=2600.4163
Epoch 210: HPWL=31069.69, Density=2600.4163
Epoch 220: HPWL=25233.89, Densi